# Simulated Annealing for Microsimulation

We apply the simulated annealing algorithm to a **microsimulation** task in this short example. Microsimulation can be used to combine individual-level and aggregate datasets that are available at different spatial resolutions in order to produce a synthetic population where every individual within has a detailed set of characteristics.

This may occur in a scenario where individual-level data is available over a large area (e.g. samples from the census), but the data is needed on a smaller spatial scale. If the numbers of people with certain properties are known at a finer spatial resolution from another data source, it is possible to generate a synthetic population at a smaller spatial scale where the properties of the people within are consistent with the second data source.

The synthetic population generated by microsimulation techniques can be used in further simulations, such as agent-based models, where the characteristics of each simulated individual in the population must be known.

In [ ]:
import numpy as np
import os
import pandas as pd
from simanneal import Annealer

## SimpleWorld

We'll work with a very simple dataset, "SimpleWorld", for the first part of this demonstration. The example data comes from *Spatial Microsimulation with R*, by Robin Lovelace and Morgane Dumont.

### Load the data

We'll first load two files that contain counts of people with certain attributes who reside in three separate regions. These attributes - here, age and sex - form the **constraints** of our problem.

In the **age** dataset, we have counts of people aged below 50, and 50 and above. The three rows of the table correspond to the three different regions. We can see that there are slightly more younger people in regions 0 and 2, and many more older people in region 1.

In [ ]:
age = pd.read_csv(os.path.join(os.getcwd(), "..", "..", "datasets", "SimpleWorld", "age.csv"))
age

The **sex** dataset is similarly formatted. The number of males and females are close two equal in regions 0 and 1, while there are far more females than males in region 2.

We would expect our synthetic dataset to reflect these characteristics to give younger population with more females in region 2, and so on for the other regions.

In [ ]:
sex = pd.read_csv(os.path.join(os.getcwd(), "..", "..", "datasets", "SimpleWorld", "sex.csv"))
sex

We also have some individual-level data. Each individual has properties which match those in the constraints, along with other properties which are of interest to us. In this example, we would like to generate a synthetic population for each of the three regions where the individuals within have an exact age and an income.

In [ ]:
ind = pd.read_csv(os.path.join(os.getcwd(), "..", "..", "datasets", "SimpleWorld", "ind-full.csv"))
ind

### Prepare the optimisation problem

In this example, we use an implementation of a simulated annealing algorithm from the `simanneal` package. We use this implementation as the one in the widely-used `scipy` package is deprecated, and both it and subsequent implementations of similar algorithms in `scipy` are not able to accept discrete choices for the inputs.

We initialise the class with the constraints a list of the individuals and their attributes. We also define two functions in the class:
- `move()` Selects one person in the synthetic population and swaps their ID to an alternative from the population of individuals.
- `energy()` Computes the total absolute error, which we want to minimise during the optimisation process. The error is defined as
$$ \sum_c\sum_o |S_{c,o} - E_{c,o}| $$
where $c$ is the constraint type, $o$ is the option for that constraint, and $S$ and $E$ are the counts of these combinations of $c$ and $o$ in the synthetic and expected (constraint) populations respectively.

In [ ]:
class MicrosimulationOptimiser(Annealer):
    """A class for performing simulated annealing on the microsimulation problem"""
    
    def __init__(self, inds, *cons, logs=False, pop_size=None):
        
        self.logs = logs
        
        # Check that the constraints are consistent - must contain same number of individuals
        for i, con in enumerate(cons):
            if i == 0:
                population_count = sum(con)
            else:
                assert sum(con) == population_count
                
        if pop_size is None:
            self.n_synth_individs = population_count
        else:
            self.n_synth_individs = pop_size
    
        self.individs = inds       # Use term individs rather than inds to avoid confusion with plural of id!
        self.cons = cons
        
        # Give each synthetic individual a random ID from the seed individual dataset, then set up the optimiser class
        state = np.random.choice(range(self.individs.shape[0]), self.n_synth_individs)
        super(MicrosimulationOptimiser, self).__init__(state)
    
    def move(self):
        
        # Select individual to be changed
        index_change = np.random.randint(self.n_synth_individs)
        
        if self.logs:
            print("Changing individual {}; was {}: \t {}".format(index_change, self.state[index_change], self.state))
        
        # Give this individual a new id, which must be different from its current id
        new_id = np.random.choice(self.individs.shape[0]-1)
        if new_id < self.state[index_change]:
            self.state[index_change] = new_id
        else:
            self.state[index_change] = new_id + 1
            
        if self.logs:
            print("now {}: \t\t\t\t {}".format(self.state[index_change], self.state))
        
    def energy(self):
        
        # Generate the synthetic population: state tells us which individuals are present by their index in inds
        synthetic_individs = self.individs[self.state,]
        
        total_abs_error = 0
        
        # Count up how many individuals there are with each property 
        # and compare to the counts in the constraint tables
        for s_individs, con in zip(synthetic_individs.transpose(), self.cons):
            
            # Count how many simulated people have each option for this constraint
            con_id, con_id_counts = np.unique(s_individs, return_counts=True)
            
            # Some options might be missing - add their label and frequency (0) to the list manually
            missing = np.where(np.isin(range(con.shape[0]), con_id, invert=True))
            if missing[0].size != 0:
                
                if self.logs:
                    print("Constraint IDs {} are missing in constraint table {}".format(missing[0], con_id))
                    for i, c in zip(con_id, con_id_counts):
                        print("[{}, {}]".format(i, c))
                
                con_id = np.append(con_id, missing[0])
                con_id_counts = np.append(con_id_counts, np.zeros_like(missing[0]))

                if self.logs:
                    print("Added missing constraint IDs:")
                    for i, c in zip(con_id, con_id_counts):
                        print("[{}, {}]".format(i, c))
                    
                # Sort the resulting counts array
                con_id_counts = con_id_counts[np.argsort(con_id)]
                
                if self.logs:
                    con_id = con_id[np.argsort(con_id)]
                    print("Sorted constraints:")
                    for i, c in zip(con_id, con_id_counts):
                        print("[{}, {}]".format(i, c))
                    
            abs_error = np.sum(np.abs(con - con_id_counts))
            total_abs_error += abs_error

            if self.logs:
                print("Constraint counts: {}".format(con))
                print("Synthetic counts: {}".format(con_id_counts))
                print("Error: {}".format(abs_error))
            
        return total_abs_error

Before we run the optimisation process, we need to reformat the table of individuals. Our seed population should have one row per person, one column for each of the constraints. The values in each column indicate which of the options for that constraint each person has.

In [ ]:
age_conditions = [ind["age"] < 50, ind["age"] >= 50]
sex_conditions = [ind["sex"] == "m", ind["sex"] == "f"]

# Now create an array that holds the individuals and how their properties correspond to the constraints
ind_array = np.array([np.select(age_conditions, range(len(age_conditions)), default=None),
                      np.select(sex_conditions, range(len(sex_conditions)), default=None)]).transpose()

print("Seed population:\n {}".format(ind_array))

The first column of the array indicates the age band of each person in the seed population: <50 (0) or >=50 (1). The second column indicates the sex: male (0) or female (1). The indices in this array correspond to the column used for that characteristic in each of the constraint arrays.

Before we start, we'll set a random seed for reproducibility and choose the region for which we want to generate a population.

In [ ]:
np.random.seed(1)
region = 0

Now, we can prepare the optimiser. We provide the following arguments:
- An array of individuals as shown above.
- Arrays containing the constraints (also converted to numpy arrays). The order of the constraints corresponds to the column ordering in the first argument.

When we view the initial state, we can see that we have an array of individuals (by default, the same number given in the constraints) with a random set of IDs, which correspond to individuals from the seed population. The optimiser will change the IDs in this array until it generates a population which is consistent with (or as close as possible to) the constraints we specified earlier.

In [ ]:
opt = MicrosimulationOptimiser(ind_array, age.to_numpy()[region], sex.to_numpy()[region])
print("Initial state:", opt.state)

Now, let's run the simulated annealing algorithm. We change `Tmax` and `Tmin` from their defaults of 25,000 and 2.5 as the documentation for the `simanneal` package recommends ~98% acceptance of moves at `Tmax` and close to 0% improvement at `Tmin`. These changes were made manually, and will need revising for other datasets. (In this case, the defaults do still give reatively good results - comment out the next two lines to try.)

In [ ]:
opt.Tmax = 100
opt.Tmin = 0.1
population_ids, error = opt.anneal()

The outputs from the annealing algorithm are a list of IDs, corresponding to individuals from our sample population who are included in our synthetic population, and the total absolute error in the population. If it is zero, our population should exactly fulfil the constraints we specified earlier.

In [ ]:
synthetic_population = ind.iloc[population_ids]
synthetic_population

Finally, let's compare the properties of our synthetic population with the original. The properties that we used for constraints are age (under/over 50) and sex.

In [ ]:
print("Region {}".format(region))

print("\nAge")
print("Under 50: expected {}, got {}".format(age["a0.49"].iloc[region], 
                                             len(synthetic_population[synthetic_population["age"] < 50])))
print("50 and over: expected {}, got {}".format(age["a.50+"].iloc[region], 
                                                len(synthetic_population[synthetic_population["age"] >= 50])))

print("\nSex")
print("Males: expected {}, got {}".format(sex["m"].iloc[region], 
                                          len(synthetic_population[synthetic_population["sex"] == "m"])))
print("Females: expected {}, got {}".format(sex["f"].iloc[region], 
                                            len(synthetic_population[synthetic_population["sex"] == "f"])))